# Packages

In [2]:




# from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold, cross_validate
# from sklearn.model_selection import learning_curve, validation_curve, cross_val_score, ShuffleSplit
# from sklearn.pipeline import Pipeline, make_pipeline

# from sklearn.impute import SimpleImputer, KNNImputer
# from sklearn.preprocessing import RobustScaler, OneHotEncoder, LabelEncoder
# from sklearn.compose import ColumnTransformer, make_column_selector,make_column_transformer

# from sklearn.metrics import classification_report, roc_curve, brier_score_loss, roc_auc_score, f1_score
# from sklearn.metrics import make_scorer, recall_score, matthews_corrcoef, fbeta_score
# from sklearn import metrics

# from sklearn.base import BaseEstimator, TransformerMixin

# from sklearn.experimental import enable_iterative_imputer  # noqa
# from sklearn.impute import IterativeImputer

# from sklearn.calibration import calibration_curve, CalibratedClassifierCV

# import category_encoders as ce

# from functools import partial
# import plotly
# import optuna

# from missingno import missingno as mn

# import numpy as np
# import matplotlib.pyplot as plt

# from sklearn import svm, datasets
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
# from sklearn.utils.multiclass import unique_labels

# from sklearn.feature_selection import RFECV
# from sklearn.ensemble import ExtraTreesRegressor

#========================
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import time

#for pipeline viz
# from sklearn import set_config                      # to change the display
# from sklearn.utils import estimator_html_repr       # to save the diagram into HTML format
# from IPython.core.display import display, HTML      # to visualize pipeline

#do we need these with pickles?
# import lightgbm as lgb
# from sklearn.ensemble import RandomForestClassifier


from copy import deepcopy

import pickle # changed to pickle from pickle5

from datetime import datetime

# set_config(display='diagram')


%matplotlib inline

In [3]:
pd.set_option('display.max_columns', 1000)

# Functions

In [ ]:
def fix_col_names(df):
    '''
    takes a df and converts column names to lowercase underscore seperated
    '''
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    return df



# PARAMETERS

# These must be as lists
SEGMENT = ['Secondary','SGB','STM'] 
FITYEARS = [2019,2021]

#for final graphs and calcs
RETENT_YEARS = [2019,2021]
CLV_YEAR = [2021]


In [ ]:
# change the localand drive path for the performance analysis project
LOCAL_DAT_PATH = r'C:\Users\rhead\Documents\local_ds\Model_refreshes\eagles\\'

DRIVE_PATH = r'C:\Users\rhead\Documents\local_ds\Model_refreshes\eagles\\'

DT_STAMP = datetime.now().strftime("%y-%m-%d_%I%M%p")
DT_STAMP

## Check connection
* if paths prints, good-to-go!  If not, check to make sure you are connected

In [ ]:
for x in os.walk(DRIVE_PATH):
    print(x[0])
    break

# Load Data

In [ ]:
## the data loaded here should match with what was loaded in the previous communication notebook
dat = pd.read_csv(LOCAL_DAT_PATH+r'EaglesQA_Training_data_0117.csv')

dat = fix_col_names(dat)

dat.shape

# Segmentation

In [ ]:
dat['customersegment'].value_counts()

In [ ]:
print(dat.shape)
dat = dat[dat['customersegment'].isin(SEGMENT)]
print(dat.shape)

dat.customersegment.value_counts()

In [ ]:
#preshuffle the data, mixing multiple years together
#create copy to only need to load once (be aware it doubles Python memory reserve) 
df = dat.sample(frac=1,random_state=222).reset_index(drop=True).copy()

## Encode Target

In [ ]:
Segment_num_dct = {'STM': 3
                   , 'SGB': 2
                   , 'Secondary': 1
                   , 'Churn': 0}

df['target'] = df['customersegmentfollowingyear'].map(Segment_num_dct)
df.target.value_counts()

In [ ]:
target = 'target'

df[target] = df[target].astype(int)

target_labels = list(set(df[target].unique())) #used to ensure correct plots later
print('target_labels', target_labels)
print('target nulls:',df[target].isnull().sum())

# Percent of labels
np.round(df[target].value_counts()/len(df[target])*100,3)

### Datetime

In [ ]:
# Any date columns needing to be coverted to datetime

ENCODE_DATES = False
if ENCODE_DATES:

    def process_time_features(df):
        date_cols = [x for x in df.columns if 'date' in x]
        print(date_cols)
        for name in date_cols:
            df[name] = pd.to_datetime(df[name])

        df.drop(date_cols,axis=1,inplace=True) #note this is dropping, comment out to keep
        
        return df

    df = process_time_features(df)

### Ordinal

# Any ordinal features in the dataset

ENCODE_ORD = False
if ENCODE_ORD:
    
    def encode_ordinal_features(df):

#         df['ownrent'] = df['ownrent'].map({ 'Definite Renter': 0
#                                             , 'Probable Renter': 0
#                                             , 'Definite Owner': 1
#                                             , 'Probable Owner': 1
#                                             })
#         df['education'] = df['education'].map({'Vocational/Technical School': 0
#                                                , 'High School': 1
#                                                , 'College': 2
#                                                , 'Graduate School': 3
#                                               })
#         df['maritalstatus'] = df['maritalstatus'].map({'Definitely Single': 1
#                                            , 'Possibly Single':1
#                                            , 'Possibly Married':2
#                                            , 'Definitely Married':2
#                                           })
#         df['householdincome'] = df['householdincome'].map({'$0-$15,000': 0
#                                                  ,'$15,001-$20,000': 1
#                                                  ,'$20,001-$30,000': 2
#                                                  ,'$30,001-$40,000': 3
#                                                  ,'$40,001-$50,000': 4
#                                                  ,'$50,001-$60,000': 5
#                                                  ,'$60,001-$75,000': 6
#                                                  ,'$75,001-$100,000': 7
#                                                  ,'$100,001-$125,000': 8
#                                                  ,'$125,001-$150,000': 9
#                                                  ,'$150,001+': 10
#                                                 })
#         df['demographicsgender'] = df['demographicsgender'].map({'Male': -1
#                                            , 'Neutral':1
#                                            , 'Female':2
#                                           })
        return df

    df = encode_ordinal_features(df)

### Categorical - Address Long Tail
* CLV_GRAPH Note - Only copy the final dictionaries over to the notebook


In [ ]:
# Check label counts of categorical features
#TODO - CREATE CLASS OF PREPROCESS FUNCTON TO INCLUDE IN PIPELINE

cat_ls = [

'acquisitionsource',

]

for c in cat_ls:
    print('================================================')
    print(c)
    print('________________________________________________')
    print(df[c].value_counts())

# =========== Setup ======================

## Filter Out SeasonYears

In [ ]:
get_years = list(set(FITYEARS+RETENT_YEARS+CLV_YEAR))

#create renewed flag
print(df.shape)
df = df[df['seasonyear'].isin(get_years)].copy()
print(df.shape)

## Renewed

In [ ]:
print('\n distribution of customersegment by year \n','_'*30)
pd.crosstab(df.customersegment, df.seasonyear, normalize='columns')

In [ ]:
#calculate renew
df['renewed'] = (df['customersegment'] == df['customersegmentfollowingyear']).astype(int)

In [ ]:
print('\n actual renewal rate by segment and by year \n','_'*30)
pd.crosstab(df.customersegment, df.seasonyear,aggfunc='mean', values=df.renewed)


# ------------------------- Model Predictions ----------

## Select Variables for Each Model

See model prep notebook for feature selection and model testing

In [ ]:
### Select Final variables for Models
stm_variables = [
 'acquisitionsource',
 'attendancerateyoydelta',
 'avgprimaryeaglesspend',
 'avgprimarypurchasedaysbeforeevent',
 'avgpurchasedaysbeforeevent',
 'avgsecondaryeaglesspend',
 'avgsecondarypurchasedaysbeforeevent',
 'avgsecondaryrevenue',
 'distancetovenue',
 'eagleseventattendancerate',
 'eaglesotherattendance',
 'eaglesseatsattendedrate',
 'eagleswinningattendanceregseasonrate',
 'email_clickrate_1yr',
 'email_clickrate_30days',
 'email_clickrate_30to60days',
 'email_clickrate_60days',
 'email_clickrate_60to90days',
 'email_clickrate_90days',
 'email_clickrate_regseason',
 'email_openrate_1yr',
 'email_openrate_30days',
 'email_openrate_30to60days',
 'email_openrate_60days',
 'email_openrate_60to90days',
 'email_openrate_90days',
 'email_openrate_regseason',
 'lffattendance',
 'lostpercent',
 'marketingattendance',
 'membersince',
 'nextYearSTMContractEnd',
 'noneaglesgamesattendance',
 'nonplaneaglesprimaryspend',
 'primaryeaglesspend',
 'primarypostseasoneaglesspend',
 'realizedvalue',
 'secondaryeaglesspend',
 'spendyoy',
 'suiteeventsattendance',
 'templeattendance',
 'totalsecondaryrevenue',
 'utilizationrate',
 'ytd_eaglesotherrevenue',
 'ytd_lffrevenue',
 'ytd_marketingrevenue',
 'ytd_noneaglesrevenue',
 'ytd_suiteeventsrevenue',
 'ytd_templerevenue'
]

In [ ]:
# Create dataset with variables for modeling each customer type
df_stm = df[df['customersegment'] == 'STM'].copy()


## Read In Models

In [ ]:
BUILD_PATH = DRIVE_PATH
print(BUILD_PATH)

#get list of files in directory
os.listdir(BUILD_PATH)

### STM Model

In [ ]:
#TODO: Consolidate to one pickle file
STM_MODEL_FILENAME = r'2019_2021_eagles_finalized_clv_STM_model_23-02-15_0103PM.p'

In [ ]:
with open(BUILD_PATH+STM_MODEL_FILENAME, 'rb') as input_file:
    stm_model = pickle.load(input_file)

### Change the n_jobs value
In the loaded environment the pickle files created in an older environment were not able to load properly and errored out on the n_jobs parameter as it was not able to parallelize the model. We had to make this bandaid fix to run the model in the new environment by changing the n_jobs parameter to 1 from -1

In [ ]:
# Print the pipeline to understand its structure
print(stm_model)

# Assuming the preprocessing step is named 'preprocess' and the classifier is named 'classifier'
preprocess_pipeline = stm_model.named_steps['preprocess_pipeline']
calibrated_classifier = stm_model.named_steps['calib_clf']

# Check if the preprocess pipeline and classifier have the n_jobs parameter
# Change n_jobs to 1 if it exists
if hasattr(preprocess_pipeline, 'n_jobs'):
    preprocess_pipeline.n_jobs = 1
    print(f"n_jobs in the preprocess pipeline is now set to: {preprocess_pipeline.n_jobs}")

if hasattr(calibrated_classifier, 'n_jobs'):
    calibrated_classifier.n_jobs = 1
    print(f"n_jobs in the classifier is now set to: {calibrated_classifier.n_jobs}")
    
# print(stm_model)

# Optionally, save the modified pipeline back to a pickle file
with open(BUILD_PATH+'2019_2021_eagles_finalized_clv_STM_model_7312024.p', 'wb') as file:
    pickle.dump(stm_model, file)

In [ ]:
#load the model again after changing the parameter
with open(BUILD_PATH+'2019_2021_eagles_finalized_clv_STM_model_7312024.p', 'rb') as input_file:
    stm_model = pickle.load(input_file)

## Predict on STM Data

In [ ]:
pred_labels = ['ProbChurn', 'ProbSecondary', 'ProbSGB', 'ProbSTM']

In [ ]:
# STM
predictions_stm = stm_model.predict_proba(df_stm[stm_variables])
predictions_stm_df = pd.DataFrame(predictions_stm, columns = pred_labels, index = df_stm.index)
predictions_stm_df.mean()

# ---------------------------- Retention Slides ----------

### Retention Decile Plot

#### Data:

In [ ]:

def create_final_retention_decile_graph(df_in, SEGMENT, get_data=False):
    """
    Displays retention graph for final slide deck along with additional labels for the mean
    probs and actuals. 
    """

    fig, ax = plt.subplots(figsize = (12, 6))

    ### Create Deciles
    df_in['Deciles'] = pd.Series(pd.qcut(df_in['Prob'+SEGMENT], 10, labels=False, duplicates='drop'))

    # Create subsets grouping by the different deciles
    deciles = df_in.groupby('Deciles')['Prob'+SEGMENT].mean()
    score_df = deciles.to_frame()
    score_df['Deciles'] = score_df.index + 1
    score_df.reset_index(drop=True, inplace=True)
    
    rfactual = df_in.groupby('Deciles')['renewed'].mean()
    score_df = pd.concat((score_df, rfactual), axis = 1)
    score_df.rename(columns={'Prob'+SEGMENT:"Model Prediction", 'renewed':'Actual Results'},inplace=True)
   
    #Reformat df shape to use in plot
    df_forplot = pd.melt(score_df, id_vars=['Deciles'], value_vars=['Model Prediction', 'Actual Results'], var_name='Model', value_name = 'Probability')
    
    ### Create Plot
    ax= sns.lineplot(x="Deciles", y="Probability", hue="Model",
                      palette= ['#1f497d', '#4290dc'], marker="D", data=df_forplot) #'#b3d3f1'
    plt.axhline(y=score_df['Actual Results'].mean(), color='#4290dc',ls='--')
    plt.axhline(y=score_df['Model Prediction'].mean(),color='#1f497d',ls='--')
    ax.set(ylim=(0, 1.1))
    plt.legend(loc='lower right')
    plt.show()

    #Note - Previously pasted the following data into slide    
    # score_df = score_df[['Deciles','Model Prediction','Actual Results']]
    # print('___________________________________')
    # print('data to copy into powerpoint graph')
    # display(score_df)
    print('___________________________________')
    print('mean values for header and graph')
    display(np.round(score_df[['Model Prediction','Actual Results']].mean()*100,2))

    if get_data:
        return score_df

In [ ]:
# TODO retain the season year from the initial model build communication graphs notebook that was put up in the model readout deck. For
# Eagles we compared the 2021 season year results. Hence, to calculate the Brier score for that year we will use the formula in the next cell
create_final_retention_decile_graph(df_stm_output[df_stm_output.seasonyear==2021].copy(), SEGMENT='STM', get_data=True)

## Brier Score

In [ ]:
df_brier = df_stm_output[['audienceid', 'renewed', 'ProbSTM']].copy()

In [ ]:
df_brier['brier_score_raw'] = np.subtract(df_brier.renewed, df_brier.ProbSTM)**2

In [ ]:
df_brier['brier_score_raw'].mean()